In [81]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates
import os

In [82]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base path to the desired directory on Google Drive
base_path = '/content/drive/MyDrive/Study_1_Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [84]:
def process_data(data, columns_to_remove):
    processed_data = data.drop(columns=columns_to_remove).values
    return processed_data

In [85]:
def construct_3d_array(base_dir, participants, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye):
    """
    Construct 3D array from CSV files.
    """
    num_rows = 180  # Define number of rows to keep (last 180 rows)
    arrays_3d = []

    for participant in participants:
        participant_id = f"{int(participant):02d}"  # Format participant number to two digits

        valid_simulations = []

        for simulation in simulations:
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')

            # Check if all files exist
            if all(os.path.exists(file) for file in [hr_file_path, gsr_file_path, head_file_path, eye_file_path]):
                valid_simulations.append(simulation)

        num_valid_simulations = len(valid_simulations)
        if num_valid_simulations == 0:
            continue  # Skip this participant if no valid simulations are found

        array_3d = np.zeros((num_valid_simulations, num_rows, 47)) # hr=1, gsr=1, head=15-3, eye=41-8 total columns after removing columns= 48

        for s_idx, simulation in enumerate(valid_simulations):
            # Process hr data
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            hr_data = read_csv(hr_file_path)
            processed_hr_data = process_data(hr_data, columns_to_remove_hr)
            processed_hr_data = processed_hr_data[-num_rows:]  # Keep only the last 180 rows

            # Process gsr data
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            gsr_data = read_csv(gsr_file_path)
            processed_gsr_data = process_data(gsr_data, columns_to_remove_gsr)
            processed_gsr_data = processed_gsr_data[-num_rows:]  # Keep only the last 180 rows

            # Process head data
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            head_data = read_csv(head_file_path)
            processed_head_data = process_data(head_data, columns_to_remove_head)
            processed_head_data = processed_head_data[-num_rows:]  # Keep only the last 180 rows

            # Process eye data
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')
            eye_data = read_csv(eye_file_path)
            processed_eye_data = process_data(eye_data, columns_to_remove_eye)
            processed_eye_data = processed_eye_data[-num_rows:]  # Keep only the last 180 rows

            # Combine processed data
            combined_data = np.concatenate((processed_hr_data, processed_gsr_data, processed_head_data, processed_eye_data), axis=1)

            array_3d[s_idx, :, :] = combined_data

        arrays_3d.append(array_3d)

    return arrays_3d


In [86]:
sample_size=60
# simulations_train = ['noise','bumps']
# simulations_test=['flat']
# val_indices = [4, 10, 11, 26, 28, 31, 33, 37] # for flat
# train_indices = [0, 1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 32, 34, 35, 36, 38, 39, 40, 41] # for flat


# simulations_test=['noise']
# simulations_train = ['flat','bumps']
# val_indices = [7, 15, 17, 19, 28, 31, 32, 42, 44, 48] # for noise
# train_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47] # for noise

simulations_test=['bumps']
simulations_train = ['flat','noise']
val_indices = [1, 12, 16, 18, 22, 26, 28, 37, 41] # for speedbumps
train_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 42, 43, 44] # for speedbumps

In [87]:
participants = [str(i) for i in range(1, 27)]  # Participants 101 to 123
columns_to_remove_hr = []
columns_to_remove_gsr = []
columns_to_remove_eye = ['#Frame','Time', 'Unnamed: 40','ConvergenceValid','Left_Eye_Closed','Right_Eye_Closed','LocalGazeValid','WorldGazeValid']
columns_to_remove_head = ['#Frame','Time', 'Unnamed: 14']

In [88]:
arrays_train = construct_3d_array(base_path, participants, simulations_train, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_test = construct_3d_array(base_path, participants, simulations_test, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)

In [89]:
# Concatenate arrays along the first axis
input_train = np.concatenate(arrays_train, axis=0)
input_test = np.concatenate(arrays_test, axis=0)

# Display the shape of the final concatenated 3D array
print(f"Shape of the final concatenated 3D array: {input_train.shape}")
print(f"Shape of the final concatenated 3D array: {input_test.shape}")

Shape of the final concatenated 3D array: (45, 180, 47)
Shape of the final concatenated 3D array: (23, 180, 47)


In [90]:
def calculate_total_ssq(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    n_columns = [0, 5, 6, 7, 8, 14, 15]
    o_columns = [0, 1, 2, 3, 4, 8, 10]
    d_columns = [4, 7, 9, 10, 11, 12, 13]

    # Calculate sum for each specified set of columns
    n_val = df.iloc[:, n_columns].sum(axis=1)
    o_val = df.iloc[:, o_columns].sum(axis=1)
    d_val = df.iloc[:, d_columns].sum(axis=1)

    return n_val,o_val,d_val

In [91]:
def merge_ssq_column(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          df = pd.read_csv(csv_path)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq_values.append([n_val, o_val, d_val])
          #ssq_values_participant = df.iloc[:, 0:17].values.flatten()   # Assuming SSQ values are in columns 1 to 16
          #total_ssq_values.append(ssq_values_participant)
  ssq_array = np.array(total_ssq_values)
  return ssq_array

def merge_total_ssq(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq = (n_val+o_val+d_val) * 3.74
          df = pd.read_csv(csv_path)
          df["total-ssq"] = total_ssq
          #print("csv_path: ",csv_path,"   ",total_ssq)
          total_ssq_values.append(total_ssq)
  # Create a DataFrame from the list of total SSQ values
  df_total_ssq = pd.DataFrame(total_ssq_values, columns=["total-ssq"])
  # Convert the list of total SSQ values to a NumPy array
  total_ssq_array = np.array(total_ssq_values)
  return total_ssq_array



In [92]:
output_train=merge_ssq_column(simulations_train,participants)
output_train = np.squeeze(output_train)
output_test=merge_ssq_column(simulations_test,participants)
output_test = np.squeeze(output_test)
output_train_total_ssq=merge_total_ssq(simulations_train,participants)
output_test_total_ssq=merge_total_ssq(simulations_test,participants)
output_train_total_ssq=output_train_total_ssq.reshape(-1, 1)
output_test_total_ssq=output_test_total_ssq.reshape(-1, 1)
# print(output_train.shape,output_test.shape,output_train_total_ssq.shape,output_test_total_ssq.shape)
# print(output_train)
# print(output_train_total_ssq)


In [93]:
def scale_input_data(input_train, input_test):
    # Get the shape of the input data
    num_samples_train, time_steps_train, num_features = input_train.shape
    num_samples_test, time_steps_test, _ = input_test.shape

    # Reshape the input data into 2D arrays
    flattened_train_data = input_train.reshape(-1, num_features)
    flattened_test_data = input_test.reshape(-1, num_features)

    # Initialize a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both train and test data
    scaled_train_data = scaler.fit_transform(flattened_train_data)
    scaled_test_data = scaler.transform(flattened_test_data)

    # Reshape the scaled data back to its original shape
    scaled_train_data = scaled_train_data.reshape(num_samples_train, time_steps_train, num_features)
    scaled_test_data = scaled_test_data.reshape(num_samples_test, time_steps_test, num_features)

    return scaled_train_data, scaled_test_data

def scale_target_var(target_data):
    min_val, max_val = np.min(target_data, axis=0), np.max(target_data, axis=0)
    target_data = (target_data-min_val)/(max_val-min_val)

    return target_data, min_val, max_val

In [94]:
input_train, input_test= scale_input_data(input_train[:, (60-sample_size):(180-sample_size), :], input_test[:, (60-sample_size):(180-sample_size), :])
output_train, min_val, max_val = scale_target_var(output_train)

input_val = input_train[val_indices]
input_train = input_train[train_indices]
output_val = output_train_total_ssq[val_indices]
output_train = output_train[train_indices]


In [95]:
print("input_train :", input_train.shape)
print("output_train :", output_train.shape)
print("input_val :", input_val.shape)
print("output_val :", output_val.shape)
print("input_test :", input_test.shape)
print("output_test :", output_test.shape)

input_train : (36, 120, 47)
output_train : (36, 3)
input_val : (9, 120, 47)
output_val : (9, 1)
input_test : (23, 120, 47)
output_test : (23, 3)


In [96]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, GRU, Dense, Dropout
from keras.models import Model
import numpy as np
import sklearn

def soft_parameter_sharing_regularizer(model, alpha=0.01):
    GRU_layers = [layer for layer in model.layers if isinstance(layer, GRU)]
    if not GRU_layers:
        return 0.0

    weights = [tf.convert_to_tensor(layer.weights[0]) for layer in GRU_layers]

    shared_weights = tf.reduce_mean(tf.stack(weights), axis=0)
    return alpha * tf.reduce_sum([tf.reduce_sum(tf.square(w - shared_weights)) for w in weights])



total_losses = []
for iteration in range(5):
    def get_individual_GRU(input_shape1, input_shape2):
        input_layer = Input(shape=(input_shape1, input_shape2))

        GRU_layers = []
        for _ in range(3):
            x = GRU(64, return_sequences=False)(input_layer)
            x = Dense(256, activation='relu')(x)
            x = Dropout(0.2)(x)
            GRU_layers.append(x)

        return GRU_layers, input_layer

    def get_output_model(GRU_outputs, output_shape):
        output_models = []
        for i in range(output_shape[1]):
            output_model = Sequential()
            output_model.add(Dense(256, activation='relu'))
            output_model.add(Dropout(0.2))
            output_model.add(Dense(1))
            output_model_output = output_model(GRU_outputs[i])
            output_models.append(output_model_output)
        return output_models

    # Reshape inputs
    train_input_reshaped = input_train.reshape((input_train.shape[0], input_train.shape[1], input_train.shape[2]))
    test_input_reshaped = input_test.reshape((input_test.shape[0], input_test.shape[1], input_test.shape[2]))
    val_input_reshaped = input_val.reshape((input_val.shape[0], input_val.shape[1], input_val.shape[2]))

    # Get individual GRU outputs
    GRU_outputs, input_layer = get_individual_GRU(input_train.shape[1], input_train.shape[2])

    # Create separate output models for each column
    output_models = get_output_model(GRU_outputs, output_train.shape)

    # Create combined model
    model = Model(inputs=input_layer, outputs=output_models)

    # Custom loss function
    def custom_loss(y_true, y_pred):
        mse = tf.keras.losses.mse(y_true, y_pred)
        reg_loss = soft_parameter_sharing_regularizer(model)
        return mse + reg_loss

    # Compile and train the model
    model.compile(loss=custom_loss, optimizer='adam', metrics=[['mse'] for _ in range(output_train.shape[1])])

    best_val = 1000000
    patience = 0
    best_model = None

    for k in range(200):
        model.fit(train_input_reshaped, [output_train[:, i] for i in range(output_train.shape[1])], epochs=1, batch_size=32)
        pred_val = np.array(model.predict(val_input_reshaped))
        pred_val = np.transpose(pred_val.squeeze(), (1, 0))
        print("k:", k, "patience:", patience)

        losses = []
        for i in range(pred_val.shape[0]):
            total_ssq = 0
            for j in range(3):
                total_ssq = np.sum(pred_val[i,j]*(max_val[j]-min_val[j]) + min_val[j])+total_ssq
            total_ssq = total_ssq*3.74
            output_val_ssq = output_val[i,0]
            loss = sklearn.metrics.mean_squared_error([total_ssq], [output_val_ssq], squared=False)
            losses.append(loss)
        tmp_val_loss = np.mean(losses)
        if tmp_val_loss <= best_val:
            best_val = tmp_val_loss
            patience = 0
            best_model = model
        else:
            patience += 1
            if patience > 10:
                break

    # Predict test data
    pred_test = np.array(best_model.predict(test_input_reshaped))
    pred_test = np.transpose(pred_test.squeeze(), (1, 0))

    # Evaluate the model
    pred_total_ssq = []
    for i in range(pred_test.shape[0]):
        total_ssq = 0
        for j in range(3):
            total_ssq = np.sum(pred_test[i,j]*(max_val[j]-min_val[j]) + min_val[j])+total_ssq
        total_ssq = total_ssq*3.74
        pred_total_ssq.append(total_ssq)

    # Overall Test Loss
    loss = sklearn.metrics.mean_squared_error(pred_total_ssq, output_test_total_ssq, squared=False)
    print("Test Loss:", loss)
    total_losses.append(loss)

average_loss = sum(total_losses) / len(total_losses)
total_losses.append(average_loss)
print("average_loss:", average_loss)

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - loss: 4.6191 - sequential_75_mse: 0.0450 - sequential_76_mse: 0.0343 - sequential_77_mse: 0.0379
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
k: 0 patience: 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 4.4470 - sequential_75_mse: 0.0554 - sequential_76_mse: 0.0388 - sequential_77_mse: 0.0277
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
k: 1 patience: 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 4.3490 - sequential_75_mse: 0.0863 - sequential_76_mse: 0.0494 - sequential_77_mse: 0.0540
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
k: 2 patience: 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 4.0847 - sequential_75_mse: 0.0427 - sequential_76_mse: 0.0242 - sequential_77_mse: 0.0189
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
k: 3 patience: 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - loss: 3.9271 - sequential_75_mse: 0.0333 - sequential_76_mse: 0.0250 - sequential_77_mse: 0.0260
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
k: 4 patience: 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/